# Classification d'images (MNIST) avec Keras (MLP)

## Imports




In [2]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 753.3 kB/s eta 0:00:00m eta 0:00:010:00:03
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached flatbuffers-22.9.24-py2.py3-none-any.whl (26 kB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 3.7 MB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached termcolor-2.0.1-py3-none-any.whl (5.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 896.5 kB/s eta 0:00:0031m9.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 2.7 MB/s e

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
import time

2022-10-07 13:42:48.407279: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 13:42:53.786501: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-07 13:42:53.786541: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-07 13:42:54.206670: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-07 13:42:57.000842: W tensorflow/stream_executor/platform/de

## Chargement de la base MNIST

La première étape consiste à charger les données MNIST selon la documentation de keras https://keras.io/datasets/

In [4]:
# Configuration options
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Load the data
x_train.shape

(60000, 28, 28)

## Prétraitement des données: l'entrée
- normaliser les données (pour que l'entrée  soit dans [0,1])
- remodeler les données d'entrée de sorte que les entrées `x` soient des vecteurs de taille 784, adaptés pour les RNN keras et non des images (matrices 28 x 28)


In [5]:
# Convert into greyscale

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Reshape the data - MLPs do not understand such things as '2D'.
# Reshape to 28 x 28 pixels = 784 features

x_train_flat = x_train.reshape(-1, 784)
x_train_flat.shape

(60000, 784)

## Convertir les classes cibles en categories (et non numerique)
- utiliser la fonction Keras `to_categorical()`


In [6]:
# Convertir les target classes to categorical ones
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

## Développer un MLP

Commençons par les réseaux de neurones entièrement connectés (NN) également appelés perceptrons multicouches (MLP).
Comme lors des TP précédents, le modèle séquentiel de keras sera utilisé avec des couches `Dense`

### Premier essai
Vous allez essayer de reproduire LeCun et al. 1998 dont les resultats sont disponibles sur le site Web du MNIST (http://yann.lecun.com/exdb/mnist/).
le taux d'erreur signalé est de 4,7%.
Nous commençons par créer l'architecture du MLP.
   - dans ce cas le réseau de neurones est entièrement connecté: le NN est séquentiel,
   - la première couche contient 300 neurones avec "relu" comme fonction d'activation,
   - la couche de sortie contient 10 neurones (un par classe), la fonction d'activation est: `softmax`.

In [7]:
# Set the input shape
input_shape = (784,)

model = Sequential()

model.add(Dense(300, activation='relu', input_shape=input_shape))

model.add(Dense(10, activation='softmax'))

# Créez un model Séquentiel (voir documentation Keras)

model = Sequential()

model.add(Dense(300, activation="relu", input_shape=(784,)))
model.add(Dense(10, activation="softmax"))

model.summary()

2022-10-07 13:43:27.273346: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-07 13:43:27.351398: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-07 13:43:27.351491: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vincent-perso): /proc/driver/nvidia/version does not exist
2022-10-07 13:43:27.376060: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 300)               235500    
                                                                 
 dense_3 (Dense)             (None, 10)                3010      
                                                                 
Total params: 238,510
Trainable params: 238,510
Non-trainable params: 0
_________________________________________________________________


Maintenant, vous aller définir les parametres d'apprentissage du modèle 
- La première étape ici est de définir la fonction de coût (`loss`) et l'optimiseur (`optimizer`).
    - Un choix naturel pour la fonction de perte `loss` est «catégorical_crossentropy» bien adaptée à la classification multiclasse. (La même vue dans le TP1)
    - pour l'optimiseur, choisissez une descente de gradient stochastique simple (`SGD()`).
    
    

- Lancer l'apprentissage et enregistrer le temps d'apprentissage

In [8]:
# Compilez le model et commencez l'apprentissage (avec 10 itérations)

loss = 'categorical_crossentropy'

optimizer = SGD()

start = time.time()

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

history = model.fit(x_train_flat, y_train, epochs=10, batch_size=128)

end = time.time()

print("Temps d'exécution: ", end - start)

history.history

2022-10-07 13:44:00.059281: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/10
469/469 [==============================] - 4s 4ms/step - loss: 1.1517 - accuracy: 0.7345
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.5364 - accuracy: 0.8687
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.4283 - accuracy: 0.8879
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.3797 - accuracy: 0.8977
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.3500 - accuracy: 0.9046
Epoch 6/10
469/469 [==============================] - 2s 5ms/step - loss: 0.3290 - accuracy: 0.9094
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.3127 - accuracy: 0.9128
Epoch 8/10
469/469 [==============================] - 2s 4ms/step - loss: 0.2994 - accuracy: 0.9163
Epoch 9/10
469/469 [==============================] - 2s 5ms/step - loss: 0.2880 - accuracy: 0.9195
Epoch 10/10
469/469 [==============================] - 2s 4ms/step - loss: 0.2780 - accuracy: 0.9225

{'loss': [1.1517205238342285,
  0.536420464515686,
  0.4282996654510498,
  0.3797236680984497,
  0.35003629326820374,
  0.32899972796440125,
  0.3126847743988037,
  0.29938653111457825,
  0.2880438268184662,
  0.2779831886291504],
 'accuracy': [0.734499990940094,
  0.8686833381652832,
  0.8879333138465881,
  0.8977166414260864,
  0.9046499729156494,
  0.9094499945640564,
  0.9128166437149048,
  0.9162999987602234,
  0.9194999933242798,
  0.9225333333015442]}

   - Mesurer le taux d'erreur sur la base `test`

In [ ]:
# Evaluez votre model après l'apprentissage sur la base donnée Test

#RUN YOUR CODE HERE

# Print Test Loss et Test Accuracy de votre modèle

#RUN YOUR CODE HERE

- Visualisation des résultats d'apprentissage et de test

In [ ]:
# Visualisez les courbes (historique des données d'apprentissage et de test) 


### Deuxième essai
Essayons d'ameliorer un peu l'apprentissage en introduisant un peu de punch dans notre optimiseur.
Pour ce faire, nous ajoutons un terme d'élan (`momentum = 0.9`) et une pénalité L2 (` decay = 1e-6`).

Cela se fait en remplaçant l'instruction `
optimiseur = SGD (), `
par
`
optimizer = SGD (learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = False), `

    
  - Relancer l'apprentissage sur ce modele et enregistrer le temps d'apprentissage



In [ ]:
# Mêmes étapes de création de modèle (nommez-le différemment), modifiez que l'optimiseur.

#RUN YOUR CODE HERE

   - Mesurer le taux d'erreur sur la base `test`

In [ ]:
# Compilez le model et commencez l'apprentissage (avec 10 itérations)

#RUN YOUR CODE HERE

In [ ]:
# Evaluez votre model après l'apprentissage sur la base donnée Test

#RUN YOUR CODE HERE

# Print Test Loss et Test Accuracy de votre modèle

#RUN YOUR CODE HERE

- Comparer les temps d'apprentissage et les taux d'erreur. Que remarquez-vous ?

In [ ]:
# Vos remarques ICI.

In [ ]:
# Visualisez les courbes (historique des données d'apprentissage et de test) 